# CollegeScorecard

In [10]:
import pandas as pd
df = pd.read_csv('../data/raw/College_Scorecard_Raw_Data_05192025/MERGED2023_24_PP.csv', low_memory=False)
df

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P11,GT_THRESHOLD_P11,MD_EARN_WNE_INC1_P11,MD_EARN_WNE_INC2_P11,MD_EARN_WNE_INC3_P11,MD_EARN_WNE_INDEP0_P11,MD_EARN_WNE_INDEP1_P11,MD_EARN_WNE_MALE0_P11,MD_EARN_WNE_MALE1_P11,SCORECARD_SECTOR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,49382201,4283601.0,42836.0,College Unbound - Newport/Aquidneck Island,Newport,RI,028400000,New England Commission on Higher Education,https://www.collegeunbound.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
6425,49425001,2609404.0,26094.0,Valley College - Fairlawn - School of Nursing,Fairlawn,OH,443333631,Accrediting Commission of Career Schools and C...,https://www.valley.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
6426,49501301,4247201.0,42472.0,Western Maricopa Education Center - Southwest ...,Buckeye,AZ,85326-5705,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
6427,49501302,4247202.0,42472.0,Western Maricopa Education Center - Northeast ...,Phoenix,AZ,85027-0000,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


Not much needs to be done, just updating dtypes

In [11]:
import yaml

# Load YAML
with open("../data/raw/College_Scorecard_Raw_Data_05192025/data.yaml", "r") as f:
    y = yaml.safe_load(f)

# Build source → dtype map
dtype_map = {}
for key, info in y["dictionary"].items():
    src = info.get("source")
    typ = info.get("type")

    if src and typ:
        if typ == "integer":
            dtype = "Int64"
        elif typ == "float":
            dtype = "float"
        else:
            dtype = "string"
        dtype_map[src] = dtype

for override_col in ["OPEID", "OPEID6"]:
    if override_col in df.columns:
        dtype_map[override_col] = "Int64"

# Apply dtypes only if column exists
for col, dtype in dtype_map.items():
    if col in df.columns:
        try:
            df[col] = df[col].astype(dtype)
        except Exception as e:
            print(f"Could not convert {col} to {dtype}: {e}")

Could not convert C150_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C150_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C200_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C200_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_FT4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_FTL4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_PT4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_PTL4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert TRANS_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert TRANS_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C100_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C100_L4_POOLED

## Exporting Data

In [12]:
all_years = []
for yr in range(4, 24):
    origin_file = f'MERGED20{yr:02d}_{yr+1:02d}_PP.csv'
    # destination_file = f'CollegeScorecard_{yr+2000}.csv'
    destination_file = f'CollegeScorecard_{yr+2000}.pkl'

    df = pd.read_csv(f'../data/raw/College_Scorecard_Raw_Data_05192025/{origin_file}', low_memory=False)

    for override_col in ["OPEID", "OPEID6"]:
        if override_col in df.columns:
            dtype_map[override_col] = "Int64"

    for col, dtype in dtype_map.items():
        if col in df.columns:
            try:
                df[col] = df[col].astype(dtype)
            except Exception as e:
                pass

    df = df.assign(Year=yr + 2000)
    cols = ["Year"] + [c for c in df.columns if c != "Year"]
    df = df[cols]

    # df.to_csv(f'../data/processed/{destination_file}', index=False)
    df.to_pickle(f'../data/processed/{destination_file}')
    all_years.append(df)
    print(origin_file, destination_file)

MERGED2004_05_PP.csv CollegeScorecard_2004.pkl
MERGED2005_06_PP.csv CollegeScorecard_2005.pkl
MERGED2006_07_PP.csv CollegeScorecard_2006.pkl
MERGED2007_08_PP.csv CollegeScorecard_2007.pkl
MERGED2008_09_PP.csv CollegeScorecard_2008.pkl
MERGED2009_10_PP.csv CollegeScorecard_2009.pkl
MERGED2010_11_PP.csv CollegeScorecard_2010.pkl
MERGED2011_12_PP.csv CollegeScorecard_2011.pkl
MERGED2012_13_PP.csv CollegeScorecard_2012.pkl
MERGED2013_14_PP.csv CollegeScorecard_2013.pkl
MERGED2014_15_PP.csv CollegeScorecard_2014.pkl
MERGED2015_16_PP.csv CollegeScorecard_2015.pkl
MERGED2016_17_PP.csv CollegeScorecard_2016.pkl
MERGED2017_18_PP.csv CollegeScorecard_2017.pkl
MERGED2018_19_PP.csv CollegeScorecard_2018.pkl
MERGED2019_20_PP.csv CollegeScorecard_2019.pkl
MERGED2020_21_PP.csv CollegeScorecard_2020.pkl
MERGED2021_22_PP.csv CollegeScorecard_2021.pkl
MERGED2022_23_PP.csv CollegeScorecard_2022.pkl
MERGED2023_24_PP.csv CollegeScorecard_2023.pkl


In [19]:
merged_df = pd.concat(all_years, ignore_index=True)

In [20]:
constants = ['Year', 'UNITID', 'INSTNM', 'LONGITUDE', 'LATITUDE', 'HBCU']
constant_dict = {
    'CITY': 'City',
    'REGION': 'Region Name',
    'SCH_DEG': 'Predominant Degree',
    'HIGHDEG': 'Highest Degree',
    'HSI': 'Hispanic Serving College',
    'CONTROL_PEPS': 'School Type',
}
variable_dict = {
    'ACTENMID': 'Median ACT English',
    'ACTMTMID': 'Median ACT Math',
    'ACTWRMID': 'Median ACT Writing',
    'ACTCMMID': 'Median ACT Score',
    'SAT_AVG': 'Average SAT Score',
    'TUITIONFEE_IN': 'In-State Tuition',
    'TUITIONFEE_OUT': 'Out-of-State Tuition',
    'COSTT4_A': 'Estimated Cost of Attendence',
    'ADM_RATE': 'Admissions Rate',
    'AVGFACSAL': 'Average Faculty Salary',
    'MD_EARN_WNE_P10': 'Median Earnings after 10yrs', #f[test] = pd.to_numeric(df[test], errors='coerce')
    # 'COUNT_NWNE_P10': 'Number of Students Unemployed after 10yrs', # should adjust to proportion
    'UG': 'Number of Undergrad Students',
    'GRADS': 'Number of Grad Students',
    'STUFACR': 'Student to Faculty Ratio',
    'ROOMBOARD_OFF': 'Off-Campus Room and Board'
}

keep_cols = (
    constants
    + list(constant_dict.keys())
    + list(variable_dict.keys())
)
merged_df = merged_df[ [col for col in keep_cols if col in merged_df.columns] ]
rename_map = {}
rename_map.update(constant_dict)
rename_map.update(variable_dict)
merged_df = merged_df.rename(columns=rename_map)
merged_df

,Year,UNITID,INSTNM,LONGITUDE,LATITUDE,HBCU,City,Region Name,Predominant Degree,Highest Degree,...,In-State Tuition,Out-of-State Tuition,Estimated Cost of Attendence,Admissions Rate,Average Faculty Salary,Median Earnings after 10yrs,Number of Undergrad Students,Number of Grad Students,Student to Faculty Ratio,Off-Campus Room and Board
0,2004,100636,Community College of the Air Force,NaN,NaN,<NA>,Montgomery,0,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>
1,2004,100654,Alabama A & M University,NaN,NaN,<NA>,Normal,5,3,4,...,4420,8320,<NA>,0.4751,5427,NaN,<NA>,1232,NaN,4770
2,2004,100663,University of Alabama at Birmingham,NaN,NaN,<NA>,Birmingham,5,3,4,...,4662,10422,<NA>,0.8592,7398,NaN,<NA>,4275,NaN,8823
3,2004,100690,Amridge University,NaN,NaN,<NA>,Montgomery,5,3,4,...,10040,10040,<NA>,NaN,3043,NaN,<NA>,253,NaN,7782
4,2004,100706,University of Alabama in Huntsville,NaN,NaN,<NA>,Huntsville,5,3,4,...,4516,9518,<NA>,0.8944,6721,NaN,<NA>,1513,NaN,6480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142202,2023,49382201,College Unbound - Newport/Aquidneck Island,NaN,NaN,<NA>,Newport,1,<NA>,0,...,10488,10488,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>
142203,2023,49425001,Valley College - Fairlawn - School of Nursing,NaN,NaN,<NA>,Fairlawn,3,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>
142204,2023,49501301,Western Maricopa Education Center - Southwest ...,NaN,NaN,<NA>,Buckeye,6,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>
142205,2023,49501302,Western Maricopa Education Center - Northeast ...,NaN,NaN,<NA>,Phoenix,6,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>


In [21]:
columns_to_fill = (
    ['LONGITUDE', 'LATITUDE', 'HBCU']
    + list(constant_dict.values())
)
for col in columns_to_fill:
    # Fill NaNs for each institution with the non-NaN value for that institution
    # I may have caused some academic metric to be inaccurate with this
    merged_df[col] = merged_df.groupby('UNITID')[col].transform(lambda x: x.ffill().bfill())
    print(col)

ipeds_region = {
    1: "New England",
    2: "Mid East",
    3: "Great Lakes",
    4: "Plains",
    5: "Southeast",
    6: "Southwest",
    7: "Rocky Mountains",
    8: "Far West",
    9: "Outlying Areas"
}
merged_df['Region Name'] = merged_df['Region Name'].map(ipeds_region)

LONGITUDE
LATITUDE
HBCU
City
Region Name
Predominant Degree
Highest Degree
Hispanic Serving College


/var/folders/11/701plvpd5f55mzysbjhsywgw0000gn/T/ipykernel_51465/2653329216.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df[col] = merged_df.groupby('UNITID')[col].transform(lambda x: x.ffill().bfill())


School Type


In [22]:
cols_to_check = ['LONGITUDE', 'LATITUDE', 'Year', 'UNITID']
merged_df = merged_df.dropna(subset=cols_to_check)
merged_df

,Year,UNITID,INSTNM,LONGITUDE,LATITUDE,HBCU,City,Region Name,Predominant Degree,Highest Degree,...,In-State Tuition,Out-of-State Tuition,Estimated Cost of Attendence,Admissions Rate,Average Faculty Salary,Median Earnings after 10yrs,Number of Undergrad Students,Number of Grad Students,Student to Faculty Ratio,Off-Campus Room and Board
1,2004,100654,Alabama A & M University,-86.568502,34.783368,1,Normal,Southeast,3,4,...,4420,8320,<NA>,0.4751,5427,NaN,<NA>,1232,NaN,4770
2,2004,100663,University of Alabama at Birmingham,-86.799345,33.505697,0,Birmingham,Southeast,3,4,...,4662,10422,<NA>,0.8592,7398,NaN,<NA>,4275,NaN,8823
3,2004,100690,Amridge University,-86.174010,32.362609,0,Montgomery,Southeast,3,4,...,10040,10040,<NA>,NaN,3043,NaN,<NA>,253,NaN,7782
4,2004,100706,University of Alabama in Huntsville,-86.640449,34.724557,0,Huntsville,Southeast,3,4,...,4516,9518,<NA>,0.8944,6721,NaN,<NA>,1513,NaN,6480
5,2004,100724,Alabama State University,-86.295677,32.364317,1,Montgomery,Southeast,3,4,...,4008,8016,<NA>,0.5042,5231,NaN,<NA>,964,NaN,5625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141697,2023,499662,Aviation Institute of Maintenance-Phoenix,-112.010360,33.410174,0,Phoenix,Southwest,<NA>,0,...,16657,16657,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,10556
141698,2023,499671,Galen Health Institutes-Las Vegas,-115.294893,36.071855,0,Las Vegas,Far West,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>
141699,2023,499680,Stellar Career College-Crown Point,-87.401656,41.474242,0,Crown Point,Great Lakes,<NA>,2,...,<NA>,<NA>,<NA>,NaN,1500,NaN,<NA>,<NA>,15.0,<NA>
141700,2023,499705,Tulsa Welding School-Dallas Campus,-96.937120,32.836133,0,Irving,Southwest,<NA>,0,...,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,30.0,<NA>


Calculated values

In [23]:
merged_df = merged_df.copy()
merged_df['Total Enrollment'] = merged_df['Number of Undergrad Students'] + merged_df['Number of Grad Students']

In [24]:
# merged_df.to_csv(f'../data/processed/CollegeScorecard_AllYears.csv', index=False)
merged_df.to_pickle(f'../data/processed/CollegeScorecard_AllYears.pkl')